In [1]:
from bs4 import BeautifulSoup
#from urllib.request import urlopen
import pandas as pd
import numpy as np
import requests

In [2]:
top_games = pd.read_csv('TopGames.csv')
top_games

,Ranking,Name,App_id,Current Players,Peak Players,Hours Played
0,0,Counter-Strike: Global Offensive,730,468446,1119102,511641971
1,1,Dota 2,570,322235,678417,299451762
2,2,PLAYERUNKNOWN'S BATTLEGROUNDS,578080,193342,418384,131122089
3,3,Apex Legends,1172470,132505,330879,107693951
4,4,Team Fortress 2,440,81521,109540,60877771
...,...,...,...,...,...,...
12520,12520,The Westport Independent,352240,0,4,61
12521,12521,Nancy Drew: Warnings at Waverly Academy,43600,0,17,3411
12522,12522,神舞幻想 Faith of Danschant - Original Soundtrack,779660,0,0,0
12523,12523,Hentai Splash 2,1315460,0,0,0


In [3]:
game_id = top_games.App_id

In [4]:
#This function creats a list of data frames:
def get_dfs(game_id):
    dfs = []
    fake_id = []
    for i in game_id:
        try:   
            page = requests.get("https://steamcharts.com/app/" +str(i)+ "#All").text
            soup = BeautifulSoup(page,'html')
            no_player = soup.find('table',{'class':"common-table"} )
            dfs.append(pd.read_html(str(no_player))[0])
        except:
            fake_id.append(i)
            print("No ID number " + str(i) + "!")
    return dfs,fake_id

In [5]:
# DO NOT run this multiple times!!!

dfs = get_dfs(game_id)

#List of ResponseID's in craig's data, but not in Steam Charts
fake_id = dfs[1]

#List of ResponseID's in both craig's data and Steam Charts
real_id = [x for x in game_id if x not in fake_id]

No ID number 1478540!
No ID number 92!
No ID number 360870!
No ID number 370060!
No ID number 1339310!
No ID number 1208600!
No ID number 242820!
No ID number 1524240!
No ID number 504720!
No ID number 353380!
No ID number 612290!
No ID number 365900!
No ID number 258180!
No ID number 1183740!
No ID number 300630!
No ID number 3483!
No ID number 208600!
No ID number 401680!
No ID number 1437010!
No ID number 262260!
No ID number 1217050!
No ID number 553880!
No ID number 531680!
No ID number 360380!
No ID number 1594900!
No ID number 303610!
No ID number 1090800!
No ID number 385130!
No ID number 296210!
No ID number 963690!
No ID number 1384020!
No ID number 396280!
No ID number 1209990!
No ID number 1185520!
No ID number 410820!
No ID number 1096720!
No ID number 985080!
No ID number 38100!
No ID number 643270!
No ID number 395850!
No ID number 1133420!
No ID number 588690!
No ID number 559940!
No ID number 270050!
No ID number 564960!
No ID number 235520!
No ID number 289400!
No ID 

In [7]:
print('The file TopGames.csv has '+str(len(game_id))+ ' games.')
print('Among them, '+str(len(dfs[1]))+ ' games have no data in Steam Charts.')

The file TopGames.csv has 12525 games.
Among them, 2269 games have no data in Steam Charts.


In [15]:
# Make a csv file to store game IDs and names, called 'real_id.csv'.
real_id_list = [x for x in game_id if x not in dfs[1]]
name = [top_games.iloc[top_games.loc[top_games['App_id']==x].index[0]]['Name'] for x in real_id_list]
real_id = pd.DataFrame({'App_id':real_id_list,'Name':name})

In [16]:
real_id

,App_id,Name
0,730,Counter-Strike: Global Offensive
1,570,Dota 2
2,578080,PLAYERUNKNOWN'S BATTLEGROUNDS
3,1172470,Apex Legends
4,440,Team Fortress 2
...,...,...
10251,485310,Head Shot
10252,1256490,Othello Let's Go
10253,409500,Anomaly Zone - Halloween Pack
10254,352240,The Westport Independent


In [27]:
real_id_four=real_id.loc[real_id.index.repeat(4)].reset_index(drop=True)
len(real_id_four)

41024

In [40]:
# Make csv files to store 'average number of players', 'gain', '%gain' and 'peak number of players' all together.

top_games_all =[]

for i in range(int(len(real_id_four)/4)):
    top_games_all.append([1,0,0,0]+list(dfs[0][i]['Avg. Players']) )  
    top_games_all.append([0,1,0,0]+list(dfs[0][i]['Gain']) ) 
    top_games_all.append([0,0,1,0]+list(dfs[0][i]['% Gain']) ) 
    top_games_all.append([0,0,0,1]+list(dfs[0][i]['Peak Players']) ) 

columns = ['Avg_Players','Gain','Perc_Gain','Peak_Players']+list(dfs[0][0].Month)
    
games_all_df = pd.concat([real_id_four,pd.DataFrame(top_games_all,columns = columns)],axis=1)
games_all_df.to_csv('GamesDataAll.csv',index=False) 


In [41]:
games_all_df = pd.read_csv('GamesDataAll.csv')
games_all_df

,App_id,Name,Avg_Players,Gain,Perc_Gain,Peak_Players,Last 30 Days,April 2021,March 2021,February 2021,...,April 2013,March 2013,February 2013,January 2013,December 2012,November 2012,October 2012,September 2012,August 2012,July 2012
0,730,Counter-Strike: Global Offensive,1,0,0,0,695044.47,723346.52,740927.82,741013.24,...,16006.0,15959.89,17372.73,16164.22,14079.39,14134.1,10739.48,16000.75,15475.39,932.57
1,730,Counter-Strike: Global Offensive,0,1,0,0,-28302.0,-17581.31,-85.42,-2196.42,...,46.11,-1412.84,1208.51,2084.83,-54.71,3394.62,-5261.27,525.36,14542.81,-
2,730,Counter-Strike: Global Offensive,0,0,1,0,-3.91%,-2.37%,-0.01%,-0.30%,...,+0.29%,-8.13%,+7.48%,+14.81%,-0.39%,+31.61%,-32.88%,+3.39%,+1559.43%,-
3,730,Counter-Strike: Global Offensive,0,0,0,1,1119102,1148077,1198581,1123485,...,31966,32166,35238,31359,27553,50533,20850,36057,52261,1642
4,570,Dota 2,1,0,0,0,413025.27,413784.97,390412.76,404832.13,...,174835.73,180878.9,166972.97,147748.14,121924.42,101077.43,75965.44,61867.68,55768.61,52721.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41019,352240,The Westport Independent,0,0,0,1,4,4,4,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41020,43600,Nancy Drew: Warnings at Waverly Academy,1,0,0,0,4.98,5.44,6.08,6.99,...,0.55,0.72,0.54,1.09,1.27,0.63,0.4,0.67,0.78,1.37
41021,43600,Nancy Drew: Warnings at Waverly Academy,0,1,0,0,-0.5,-0.64,-0.91,-0.94,...,-0.17,0.18,-0.55,-0.18,0.64,0.23,-0.26,-0.11,-0.59,-
41022,43600,Nancy Drew: Warnings at Waverly Academy,0,0,1,0,-8.38%,-10.51%,-13.04%,-11.86%,...,-24.04%,+32.79%,-50.16%,-13.98%,+100.69%,+56.08%,-39.57%,-13.93%,-43.30%,-
